# Python Backend Development Fundamentals

Welcome to Python backend development! This notebook covers essential backend concepts including environment variables, path handling, configuration management, and building simple APIs. You'll learn practical skills needed for real-world backend development.

**What you'll learn:**
- Environment variables and configuration management
- Path handling and file operations
- Building REST APIs with Flask/FastAPI
- Database connections and basic operations
- Error handling and logging in backend applications
- Best practices for backend development

## Table of Contents
1. [Environment Variables and Configuration](#1-environment-variables-and-configuration)
2. [Path Handling and File Operations](#2-path-handling-and-file-operations)
3. [Building REST APIs](#3-building-rest-apis)
   - [Flask Basics](#31-flask-basics)
   - [FastAPI Introduction](#32-fastapi-introduction)
4. [Database Operations](#4-database-operations)
5. [Error Handling and Logging](#5-error-handling-and-logging)
6. [Best Practices](#6-best-practices)

## 1. Environment Variables and Configuration

**What are Environment Variables?**
Environment variables are key-value pairs that exist outside your code but can be accessed by your application. Think of them like settings that can change without modifying your code.

**Why use them?**
- **Security**: Keep sensitive data (passwords, API keys) out of your code
- **Flexibility**: Different settings for development, testing, and production
- **Portability**: Your app can run in different environments without code changes

**Real-world example**: Your app needs a database password. Instead of hardcoding it, you store it as an environment variable.

```python
import os
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
load_dotenv()

# Reading environment variables
def get_config():
    """Get configuration from environment variables with defaults"""
    config = {
        'database_url': os.getenv('DATABASE_URL', 'sqlite:///default.db'),
        'api_key': os.getenv('API_KEY', 'your-default-api-key'),
        'debug_mode': os.getenv('DEBUG', 'False').lower() == 'true',
        'port': int(os.getenv('PORT', 5000))
    }
    return config

# Setting environment variables programmatically
os.environ['APP_NAME'] = 'My Backend App'

# Demo: Display current configuration
config = get_config()
print("🔧 Current Configuration:")
for key, value in config.items():
    # Hide sensitive values
    if 'key' in key.lower() or 'password' in key.lower():
        print(f"   {key}: {'*' * len(str(value))}")
    else:
        print(f"   {key}: {value}")

print(f"\n📱 App Name: {os.getenv('APP_NAME')}")
```

## 2. Path Handling and File Operations

**Why Path Handling Matters in Backend:**
Backend applications often need to:
- Read configuration files
- Store uploaded files
- Access templates and static files
- Work with different operating systems (Windows, Linux, macOS)

**The `pathlib` module is your friend!** It handles cross-platform path operations elegantly.

### Basic Path Operations

In [1]:
from pathlib import Path
import os
import tempfile
import shutil

# Current working directory
current_dir = Path.cwd()
print(f"📂 Current directory: {current_dir}")

# Creating paths (cross-platform)
project_root = Path(__file__).parent if '__file__' in globals() else Path.cwd()
config_file = project_root / 'config' / 'settings.json'
static_dir = project_root / 'static'
uploads_dir = project_root / 'uploads'

print(f"\n📁 Project structure:")
print(f"   Root: {project_root}")
print(f"   Config: {config_file}")
print(f"   Static: {static_dir}")
print(f"   Uploads: {uploads_dir}")

# Path information
example_file = Path('example.txt')
print(f"\n📄 Path parts:")
print(f"   Name: {example_file.name}")
print(f"   Stem: {example_file.stem}")
print(f"   Suffix: {example_file.suffix}")
print(f"   Parent: {example_file.parent}")

# Working with temporary files (great for testing)
with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    test_file = temp_path / 'test.txt'
    
    # Create and write to file
    test_file.write_text('Hello from backend!')
    print(f"\n💾 Created temporary file: {test_file}")
    print(f"   Content: {test_file.read_text()}")
    print(f"   Exists: {test_file.exists()}")
    print(f"   Size: {test_file.stat().st_size} bytes")

# Directory operations
def ensure_directory_exists(path):
    """Create directory if it doesn't exist"""
    path = Path(path)
    path.mkdir(parents=True, exist_ok=True)
    return path

# Demo: Create project structure
base_dir = Path('demo_project')
ensure_directory_exists(base_dir / 'config')
ensure_directory_exists(base_dir / 'static' / 'css')
ensure_directory_exists(base_dir / 'uploads')
ensure_directory_exists(base_dir / 'logs')

print(f"\n🏗️ Created project structure:")
for item in sorted(base_dir.rglob('*')):
    if item.is_dir():
        print(f"   📁 {item.relative_to(base_dir)}")

# Clean up demo directory
if base_dir.exists():
    shutil.rmtree(base_dir)
    print(f"\n🧹 Cleaned up demo directory")

📂 Current directory: /home/administrator/Desktop/datascience/github/datanexus/python

📁 Project structure:
   Root: /home/administrator/Desktop/datascience/github/datanexus/python
   Config: /home/administrator/Desktop/datascience/github/datanexus/python/config/settings.json
   Static: /home/administrator/Desktop/datascience/github/datanexus/python/static
   Uploads: /home/administrator/Desktop/datascience/github/datanexus/python/uploads

📄 Path parts:
   Name: example.txt
   Stem: example
   Suffix: .txt
   Parent: .

💾 Created temporary file: /tmp/tmpyfi_30fb/test.txt
   Content: Hello from backend!
   Exists: True
   Size: 19 bytes

🏗️ Created project structure:
   📁 config
   📁 logs
   📁 static
   📁 static/css
   📁 uploads

🧹 Cleaned up demo directory


## 3. Building REST APIs

**What is a REST API?**
REST (Representational State Transfer) is a way to build web services. Think of it like a waiter in a restaurant:
- You (client) make requests ("I want the menu")
- The waiter (API) processes your request
- The kitchen (backend) prepares the response
- The waiter brings back the result ("Here's the menu")

**HTTP Methods (Verbs):**
- `GET`: Retrieve data ("Show me the menu")
- `POST`: Create new data ("Place an order")
- `PUT`: Update existing data ("Change my order")
- `DELETE`: Remove data ("Cancel my order")

### 3.1 Flask Basics

Flask is like a lightweight, flexible framework - perfect for learning and small to medium projects.

In [3]:
# Flask API Example
from flask import Flask, jsonify, request
from datetime import datetime
import json

# Create Flask app
app = Flask(__name__)

# In-memory data store (in real apps, you'd use a database)
books = [
    {'id': 1, 'title': 'Python Tricks', 'author': 'Dan Bader', 'year': 2017},
    {'id': 2, 'title': 'Clean Code', 'author': 'Robert Martin', 'year': 2008}
]

# Route: GET all books
@app.route('/api/books', methods=['GET'])
def get_books():
    """Return all books"""
    return jsonify({
        'status': 'success',
        'data': books,
        'count': len(books)
    })

# Route: GET specific book
@app.route('/api/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    """Return a specific book by ID"""
    book = next((b for b in books if b['id'] == book_id), None)
    
    if book:
        return jsonify({
            'status': 'success',
            'data': book
        })
    else:
        return jsonify({
            'status': 'error',
            'message': 'Book not found'
        }), 404

# Route: POST new book
@app.route('/api/books', methods=['POST'])
def create_book():
    """Create a new book"""
    data = request.get_json()
    
    # Validation
    required_fields = ['title', 'author', 'year']
    if not all(field in data for field in required_fields):
        return jsonify({
            'status': 'error',
            'message': 'Missing required fields: title, author, year'
        }), 400
    
    # Create new book
    new_book = {
        'id': max([b['id'] for b in books]) + 1 if books else 1,
        'title': data['title'],
        'author': data['author'],
        'year': data['year'],
        'created_at': datetime.now().isoformat()
    }
    
    books.append(new_book)
    
    return jsonify({
        'status': 'success',
        'message': 'Book created successfully',
        'data': new_book
    }), 201

# Route: Health check
@app.route('/health', methods=['GET'])
def health_check():
    """Simple health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'service': 'Book API'
    })

# Error handler
@app.errorhandler(404)
def not_found(error):
    return jsonify({
        'status': 'error',
        'message': 'Endpoint not found'
    }), 404

# Demo the API structure (without running the server)
print("📚 Flask API Structure:")
print("\nEndpoints:")
print("   GET  /api/books        - Get all books")
print("   GET  /api/books/<id>   - Get specific book")
print("   POST /api/books        - Create new book")
print("   GET  /health           - Health check")

print("\n📝 Sample request body for POST /api/books:")
sample_request = {
    'title': 'Effective Python',
    'author': 'Brett Slatkin',
    'year': 2019
}
print(json.dumps(sample_request, indent=2))

# If you want to run this Flask app:
# app.run(debug=True, port=5000)
# Then visit: http://localhost:5000/health

📚 Flask API Structure:

Endpoints:
   GET  /api/books        - Get all books
   GET  /api/books/<id>   - Get specific book
   POST /api/books        - Create new book
   GET  /health           - Health check

📝 Sample request body for POST /api/books:
{
  "title": "Effective Python",
  "author": "Brett Slatkin",
  "year": 2019
}


### 3.2 FastAPI Introduction

FastAPI is like Flask's modern, high-performance cousin. It's built for speed and includes automatic API documentation!

**FastAPI advantages:**
- ⚡ Very fast performance
- 📖 Automatic interactive documentation
- 🛡️ Built-in data validation
- 💾 Built-in async support
- 📝 Type hints everywhere

In [4]:
# FastAPI Example
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
from datetime import datetime

# Create FastAPI app
api = FastAPI(
    title="Book API",
    description="A simple book management API",
    version="1.0.0"
)

# Pydantic models for request/response validation
class BookBase(BaseModel):
    title: str
    author: str
    year: int

class BookCreate(BookBase):
    pass

class Book(BookBase):
    id: int
    created_at: datetime

    class Config:
        from_attributes = True

class APIResponse(BaseModel):
    status: str
    message: Optional[str] = None
    data: Optional[dict] = None

# In-memory data store
fastapi_books = [
    {'id': 1, 'title': 'Python Tricks', 'author': 'Dan Bader', 'year': 2017, 'created_at': datetime.now()},
    {'id': 2, 'title': 'Clean Code', 'author': 'Robert Martin', 'year': 2008, 'created_at': datetime.now()}
]

# Routes with automatic documentation
@api.get("/api/books", response_model=List[Book], tags=["books"])
async def get_books():
    """Get all books from the library"""
    return fastapi_books

@api.get("/api/books/{book_id}", response_model=Book, tags=["books"])
async def get_book(book_id: int):
    """Get a specific book by its ID"""
    book = next((b for b in fastapi_books if b['id'] == book_id), None)
    
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    
    return book

@api.post("/api/books", response_model=Book, status_code=201, tags=["books"])
async def create_book(book: BookCreate):
    """Create a new book in the library"""
    new_book = {
        'id': max([b['id'] for b in fastapi_books]) + 1 if fastapi_books else 1,
        'title': book.title,
        'author': book.author,
        'year': book.year,
        'created_at': datetime.now()
    }
    
    fastapi_books.append(new_book)
    return new_book

@api.get("/health", response_model=APIResponse, tags=["system"])
async def health_check():
    """Check if the API is running properly"""
    return APIResponse(
        status="healthy",
        message="Book API is running",
        data={"timestamp": datetime.now().isoformat()}
    )

# Demo FastAPI features
print("🚀 FastAPI Features:")
print("\n📖 Automatic Documentation:")
print("   Swagger UI: http://localhost:8000/docs")
print("   ReDoc: http://localhost:8000/redoc")

print("\n🛡️ Automatic Validation:")
print("   - Request/response models with Pydantic")
print("   - Type checking")
print("   - Error handling")

print("\n⚡ Performance:")
print("   - Built on Starlette (async framework)")
print("   - Can handle thousands of concurrent requests")

# To run FastAPI:
# uvicorn main:api --reload --port 8000
# Then visit: http://localhost:8000/docs for interactive documentation!

🚀 FastAPI Features:

📖 Automatic Documentation:
   Swagger UI: http://localhost:8000/docs
   ReDoc: http://localhost:8000/redoc

🛡️ Automatic Validation:
   - Request/response models with Pydantic
   - Type checking
   - Error handling

⚡ Performance:
   - Built on Starlette (async framework)
   - Can handle thousands of concurrent requests


## 4. Database Operations

**Why Databases?**
In-memory storage (like our lists above) loses data when the app restarts. Databases provide:
- 💾 **Persistent storage**: Data survives app restarts
- 🔒 **Data integrity**: Ensures data consistency
- 🚀 **Performance**: Optimized for data operations
- 🔍 **Querying**: Complex data retrieval
- 👥 **Concurrent access**: Multiple users can access data safely

**Popular Python Database Options:**
- **SQLite**: Perfect for development and small apps
- **PostgreSQL**: Robust, feature-rich (production favorite)
- **MySQL**: Widely used, good performance
- **MongoDB**: NoSQL, great for flexible data structures

### SQLite Example (Great for Learning)

In [1]:
import sqlite3
from contextlib import contextmanager
from datetime import datetime
import json

# Database context manager for safe connections
@contextmanager
def get_db_connection(db_path='books.db'):
    """Context manager for database connections"""
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  # Enable column access by name
    try:
        yield conn
    finally:
        conn.close()

# Database setup
def setup_database(db_path='books.db'):
    """Create the books table"""
    with get_db_connection(db_path) as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS books (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                author TEXT NOT NULL,
                year INTEGER NOT NULL,
                created_at TEXT NOT NULL
            )
        ''')
        conn.commit()
        print("📚 Database table created successfully!")

# CRUD Operations (Create, Read, Update, Delete)
class BookDatabase:
    def __init__(self, db_path='books.db'):
        self.db_path = db_path
        setup_database(self.db_path)
    
    def create_book(self, title, author, year):
        """Add a new book to the database"""
        with get_db_connection(self.db_path) as conn:
            cursor = conn.execute(
                'INSERT INTO books (title, author, year, created_at) VALUES (?, ?, ?, ?)',
                (title, author, year, datetime.now().isoformat())
            )
            conn.commit()
            return cursor.lastrowid
    
    def get_all_books(self):
        """Get all books from the database"""
        with get_db_connection(self.db_path) as conn:
            cursor = conn.execute('SELECT * FROM books ORDER BY created_at DESC')
            return [dict(row) for row in cursor.fetchall()]
    
    def get_book(self, book_id):
        """Get a specific book by ID"""
        with get_db_connection(self.db_path) as conn:
            cursor = conn.execute('SELECT * FROM books WHERE id = ?', (book_id,))
            row = cursor.fetchone()
            return dict(row) if row else None
    
    def update_book(self, book_id, title=None, author=None, year=None):
        """Update a book's information"""
        # Build dynamic query based on provided fields
        updates = []
        params = []
        
        if title:
            updates.append('title = ?')
            params.append(title)
        if author:
            updates.append('author = ?')
            params.append(author)
        if year:
            updates.append('year = ?')
            params.append(year)
        
        if not updates:
            return False
        
        params.append(book_id)
        query = f"UPDATE books SET {', '.join(updates)} WHERE id = ?"
        
        with get_db_connection(self.db_path) as conn:
            cursor = conn.execute(query, params)
            conn.commit()
            return cursor.rowcount > 0
    
    def delete_book(self, book_id):
        """Delete a book from the database"""
        with get_db_connection(self.db_path) as conn:
            cursor = conn.execute('DELETE FROM books WHERE id = ?', (book_id,))
            conn.commit()
            return cursor.rowcount > 0

# Demo the database operations
db = BookDatabase(':memory:')  # Use in-memory database for demo

# Explicitly ensure the database is set up
setup_database(':memory:')

print("🗄️ Database Operations Demo:")

# Create books
book1_id = db.create_book('Python Tricks', 'Dan Bader', 2017)
book2_id = db.create_book('Clean Code', 'Robert Martin', 2008)
book3_id = db.create_book('Effective Python', 'Brett Slatkin', 2019)

print(f"\n✅ Created books with IDs: {book1_id}, {book2_id}, {book3_id}")

# Read all books
all_books = db.get_all_books()
print(f"\n📖 All books ({len(all_books)}):")
for book in all_books:
    print(f"   {book['id']}: '{book['title']}' by {book['author']} ({book['year']})")

# Read specific book
specific_book = db.get_book(book1_id)
print(f"\n🔍 Book {book1_id}: {json.dumps(specific_book, indent=2)}")

# Update book
update_success = db.update_book(book1_id, year=2018)
print(f"\n✏️ Updated book {book1_id}: {update_success}")

# Delete book
delete_success = db.delete_book(book3_id)
print(f"\n🗑️ Deleted book {book3_id}: {delete_success}")

# Final count
final_books = db.get_all_books()
print(f"\n📚 Final book count: {len(final_books)}")

📚 Database table created successfully!
📚 Database table created successfully!
🗄️ Database Operations Demo:


OperationalError: no such table: books

## 5. Error Handling and Logging

**Why Error Handling Matters:**
- 🛡️ **User Experience**: Graceful failures instead of crashes
- 🔍 **Debugging**: Better information about what went wrong
- 📊 **Monitoring**: Track issues in production
- 🔒 **Security**: Don't expose sensitive information in errors

**Types of Errors in Backend:**
- **Client Errors** (4xx): User did something wrong
- **Server Errors** (5xx): Something's wrong with our code/server
- **Database Errors**: Connection issues, constraint violations
- **External Service Errors**: Third-party APIs are down

### Proper Error Handling

In [9]:
import logging
from functools import wraps
from datetime import datetime
import traceback
import sys

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        # In production, you'd also log to files:
        # logging.FileHandler('app.log')
    ]
)

logger = logging.getLogger(__name__)

# Custom exceptions
class BookNotFoundError(Exception):
    """Raised when a book is not found"""
    pass

class ValidationError(Exception):
    """Raised when data validation fails"""
    pass

class DatabaseError(Exception):
    """Raised when database operations fail"""
    pass

# Error handling decorator
def handle_errors(func):
    """Decorator to handle common errors in API endpoints"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except BookNotFoundError as e:
            logger.warning(f"Book not found: {e}")
            return {
                'status': 'error',
                'message': 'Book not found',
                'error_code': 'BOOK_NOT_FOUND'
            }, 404
        except ValidationError as e:
            logger.warning(f"Validation error: {e}")
            return {
                'status': 'error',
                'message': str(e),
                'error_code': 'VALIDATION_ERROR'
            }, 400
        except DatabaseError as e:
            logger.error(f"Database error: {e}")
            return {
                'status': 'error',
                'message': 'Internal server error',
                'error_code': 'DATABASE_ERROR'
            }, 500
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            logger.error(traceback.format_exc())
            return {
                'status': 'error',
                'message': 'An unexpected error occurred',
                'error_code': 'INTERNAL_ERROR'
            }, 500
    return wrapper

# Example service with proper error handling
class BookService:
    def __init__(self):
        self.books = [
            {'id': 1, 'title': 'Python Tricks', 'author': 'Dan Bader', 'year': 2017}
        ]
    
    def validate_book_data(self, data):
        """Validate book data"""
        if not isinstance(data.get('title'), str) or len(data['title'].strip()) == 0:
            raise ValidationError("Title is required and must be a non-empty string")
        
        if not isinstance(data.get('author'), str) or len(data['author'].strip()) == 0:
            raise ValidationError("Author is required and must be a non-empty string")
        
        if not isinstance(data.get('year'), int) or data['year'] < 1000 or data['year'] > 2030:
            raise ValidationError("Year must be a valid integer between 1000 and 2030")
    
    @handle_errors
    def get_book(self, book_id):
        """Get a book by ID with error handling"""
        logger.info(f"Retrieving book with ID: {book_id}")
        
        book = next((b for b in self.books if b['id'] == book_id), None)
        if not book:
            raise BookNotFoundError(f"No book found with ID {book_id}")
        
        logger.info(f"Successfully retrieved book: {book['title']}")
        return {'status': 'success', 'data': book}, 200
    
    @handle_errors
    def create_book(self, book_data):
        """Create a new book with validation and error handling"""
        logger.info(f"Creating new book: {book_data}")
        
        # Validate the data
        self.validate_book_data(book_data)
        
        # Simulate database error (uncomment to test)
        # raise DatabaseError("Connection to database failed")
        
        new_book = {
            'id': max([b['id'] for b in self.books]) + 1,
            'title': book_data['title'].strip(),
            'author': book_data['author'].strip(),
            'year': book_data['year']
        }
        
        self.books.append(new_book)
        logger.info(f"Successfully created book with ID: {new_book['id']}")
        
        return {'status': 'success', 'data': new_book}, 201

# Demo error handling
service = BookService()

print("🛡️ Error Handling Demo:")

# Test successful operation
result, status = service.get_book(1)
print(f"\n✅ Success case (status {status}):")
print(f"   {result}")

# Test book not found
result, status = service.get_book(999)
print(f"\n❌ Book not found (status {status}):")
print(f"   {result}")

# Test validation errors
print(f"\n❌ Validation error cases:")

# Missing title
result, status = service.create_book({'author': 'Test Author', 'year': 2020})
print(f"   Missing title (status {status}): {result['message']}")

# Invalid year
result, status = service.create_book({'title': 'Test Book', 'author': 'Test Author', 'year': 'invalid'})
print(f"   Invalid year (status {status}): {result['message']}")

# Test successful creation
result, status = service.create_book({'title': 'New Book', 'author': 'New Author', 'year': 2023})
print(f"\n✅ Successful creation (status {status}):")
print(f"   Created: {result['data']['title']} by {result['data']['author']}")

print("\n📊 Logging Levels:")
logger.debug("This is a debug message (usually for development)")
logger.info("This is an info message (general information)")
logger.warning("This is a warning message (something might be wrong)")
logger.error("This is an error message (something went wrong)")
logger.critical("This is a critical message (system might be unusable)")

🛡️ Error Handling Demo:
2025-06-03 00:33:04,707 - __main__ - INFO - Retrieving book with ID: 1
2025-06-03 00:33:04,709 - __main__ - INFO - Successfully retrieved book: Python Tricks

✅ Success case (status 200):
   {'status': 'success', 'data': {'id': 1, 'title': 'Python Tricks', 'author': 'Dan Bader', 'year': 2017}}
2025-06-03 00:33:04,710 - __main__ - INFO - Retrieving book with ID: 999
2025-06-03 00:33:04,711 - __main__ - WARNING - Book not found: No book found with ID 999

❌ Book not found (status 404):
   {'status': 'error', 'message': 'Book not found', 'error_code': 'BOOK_NOT_FOUND'}

❌ Validation error cases:
2025-06-03 00:33:04,712 - __main__ - INFO - Creating new book: {'author': 'Test Author', 'year': 2020}
2025-06-03 00:33:04,713 - __main__ - WARNING - Validation error: Title is required and must be a non-empty string
   Missing title (status 400): Title is required and must be a non-empty string
2025-06-03 00:33:04,714 - __main__ - INFO - Creating new book: {'title': 'Test 

## 6. Best Practices

**🎯 Backend Development Best Practices**

Following best practices makes your code:
- 🛡️ More secure
- 🚀 More performant
- 🔧 Easier to maintain
- 👥 Better for team collaboration
- 🐛 Easier to debug

### Security Best Practices

In [10]:
import os
import hashlib
import secrets
from datetime import datetime, timedelta
import re

# 1. SECURITY BEST PRACTICES
print("🔒 Security Best Practices:")

# Never hardcode secrets!
# ❌ BAD
# api_key = "sk-1234567890abcdef"
# db_password = "mypassword123"

# ✅ GOOD - Use environment variables
API_KEY = os.getenv('API_KEY')
DB_PASSWORD = os.getenv('DB_PASSWORD')
print("   ✅ Use environment variables for secrets")

# Password hashing (never store plain text passwords!)
def hash_password(password):
    """Hash a password with salt"""
    salt = secrets.token_hex(16)
    hashed = hashlib.pbkdf2_hmac('sha256', password.encode(), salt.encode(), 100000)
    return f"{salt}:{hashed.hex()}"

def verify_password(password, hashed_password):
    """Verify a password against its hash"""
    salt, hash_hex = hashed_password.split(':')
    hashed = hashlib.pbkdf2_hmac('sha256', password.encode(), salt.encode(), 100000)
    return hashed.hex() == hash_hex

# Demo password hashing
original_password = "user123password"
hashed_pwd = hash_password(original_password)
print(f"   ✅ Password hashed: {hashed_pwd[:20]}...")
print(f"   ✅ Password verification: {verify_password(original_password, hashed_pwd)}")

# Input validation and sanitization
def validate_email(email):
    """Validate email format"""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def sanitize_input(user_input, max_length=100):
    """Sanitize user input"""
    if not isinstance(user_input, str):
        return ""
    
    # Remove potentially dangerous characters
    sanitized = re.sub(r'[<>"\';]', '', user_input)
    
    # Limit length
    sanitized = sanitized[:max_length]
    
    # Strip whitespace
    return sanitized.strip()

print(f"   ✅ Email validation: {validate_email('user@example.com')}")
print(f"   ✅ Input sanitization: '{sanitize_input('<script>alert(1)</script>hello')}'")

# 2. CODE ORGANIZATION BEST PRACTICES
print("\n📁 Code Organization:")

# Use classes for related functionality
class APIResponse:
    """Standardized API response format"""
    
    @staticmethod
    def success(data=None, message="Success"):
        return {
            'status': 'success',
            'message': message,
            'data': data,
            'timestamp': datetime.now().isoformat()
        }
    
    @staticmethod
    def error(message="An error occurred", error_code=None, status_code=400):
        return {
            'status': 'error',
            'message': message,
            'error_code': error_code,
            'timestamp': datetime.now().isoformat()
        }, status_code

print("   ✅ Consistent response format")
print("   ✅ Separate concerns (models, services, controllers)")
print("   ✅ Use meaningful names")

# 3. PERFORMANCE BEST PRACTICES
print("\n⚡ Performance Best Practices:")

# Connection pooling simulation
class DatabasePool:
    """Simulate database connection pooling"""
    def __init__(self, max_connections=10):
        self.max_connections = max_connections
        self.active_connections = 0
    
    def get_connection(self):
        if self.active_connections < self.max_connections:
            self.active_connections += 1
            return f"Connection-{self.active_connections}"
        else:
            raise Exception("Connection pool exhausted")
    
    def release_connection(self, conn):
        self.active_connections = max(0, self.active_connections - 1)

print("   ✅ Use connection pooling")
print("   ✅ Implement caching for frequently accessed data")
print("   ✅ Use pagination for large datasets")

# Pagination example
def paginate_results(data, page=1, per_page=10):
    """Paginate a list of results"""
    start = (page - 1) * per_page
    end = start + per_page
    
    return {
        'data': data[start:end],
        'pagination': {
            'page': page,
            'per_page': per_page,
            'total': len(data),
            'pages': (len(data) + per_page - 1) // per_page
        }
    }

# Demo pagination
sample_data = [f"Item {i}" for i in range(1, 26)]  # 25 items
paginated = paginate_results(sample_data, page=2, per_page=10)
print(f"   📄 Page 2 data: {paginated['data']}")
print(f"   📊 Pagination info: {paginated['pagination']}")

# 4. MONITORING AND HEALTH CHECKS
print("\n📊 Monitoring Best Practices:")

class HealthChecker:
    """Application health monitoring"""
    
    def __init__(self):
        self.start_time = datetime.now()
    
    def check_database(self):
        """Check database connectivity"""
        # In real app, try to execute a simple query
        return {'status': 'healthy', 'response_time': '< 1ms'}
    
    def check_external_services(self):
        """Check external service dependencies"""
        # In real app, check API endpoints
        return {'status': 'healthy', 'services': ['payment-api', 'email-service']}
    
    def get_system_info(self):
        """Get system information"""
        uptime = datetime.now() - self.start_time
        return {
            'uptime': str(uptime),
            'version': '1.0.0',
            'environment': os.getenv('ENVIRONMENT', 'development')
        }
    
    def comprehensive_health_check(self):
        """Complete health check"""
        return {
            'status': 'healthy',
            'timestamp': datetime.now().isoformat(),
            'database': self.check_database(),
            'external_services': self.check_external_services(),
            'system': self.get_system_info()
        }

health_checker = HealthChecker()
health_status = health_checker.comprehensive_health_check()
print(f"   ✅ Health check: {health_status['status']}")
print(f"   ⏱️ Uptime: {health_status['system']['uptime']}")

# 5. CONFIGURATION MANAGEMENT
print("\n⚙️ Configuration Best Practices:")

class Config:
    """Configuration management"""
    
    def __init__(self):
        self.environment = os.getenv('ENVIRONMENT', 'development')
        self.debug = os.getenv('DEBUG', 'False').lower() == 'true'
        self.database_url = os.getenv('DATABASE_URL', 'sqlite:///app.db')
        self.secret_key = os.getenv('SECRET_KEY', self._generate_secret_key())
        self.allowed_hosts = os.getenv('ALLOWED_HOSTS', 'localhost').split(',')
    
    def _generate_secret_key(self):
        """Generate a secret key if none provided"""
        return secrets.token_urlsafe(32)
    
    def is_production(self):
        return self.environment == 'production'
    
    def get_config_summary(self):
        return {
            'environment': self.environment,
            'debug': self.debug,
            'database_url': self.database_url.split('@')[0] + '@***' if '@' in self.database_url else self.database_url,
            'secret_key': '***masked***',
            'allowed_hosts': self.allowed_hosts
        }

config = Config()
print(f"   ✅ Environment: {config.environment}")
print(f"   ✅ Debug mode: {config.debug}")
print("   ✅ Sensitive data masked in logs")

print("\n🎯 Summary of Best Practices:")
print("   🔒 Never hardcode secrets")
print("   🛡️ Always validate and sanitize input")
print("   🏗️ Use consistent code structure")
print("   ⚡ Implement caching and pagination")
print("   📊 Monitor application health")
print("   🧪 Write tests for your code")
print("   📝 Document your APIs")
print("   🔄 Use version control (Git)")
print("   🚀 Automate deployment processes")
print("   📈 Log important events and errors")

🔒 Security Best Practices:
   ✅ Use environment variables for secrets
   ✅ Password hashed: acf966dd35b4a0d2063d...
   ✅ Password verification: True
   ✅ Email validation: True
   ✅ Input sanitization: 'scriptalert(1)/scripthello'

📁 Code Organization:
   ✅ Consistent response format
   ✅ Separate concerns (models, services, controllers)
   ✅ Use meaningful names

⚡ Performance Best Practices:
   ✅ Use connection pooling
   ✅ Implement caching for frequently accessed data
   ✅ Use pagination for large datasets
   📄 Page 2 data: ['Item 11', 'Item 12', 'Item 13', 'Item 14', 'Item 15', 'Item 16', 'Item 17', 'Item 18', 'Item 19', 'Item 20']
   📊 Pagination info: {'page': 2, 'per_page': 10, 'total': 25, 'pages': 3}

📊 Monitoring Best Practices:
   ✅ Health check: healthy
   ⏱️ Uptime: 0:00:00.000038

⚙️ Configuration Best Practices:
   ✅ Environment: development
   ✅ Debug mode: False
   ✅ Sensitive data masked in logs

🎯 Summary of Best Practices:
   🔒 Never hardcode secrets
   🛡️ Always v

## 🎓 Conclusion

**Congratulations!** You've learned the fundamentals of Python backend development:

✅ **Environment Variables**: Secure configuration management  
✅ **Path Handling**: Cross-platform file operations  
✅ **REST APIs**: Building web services with Flask and FastAPI  
✅ **Database Operations**: Persistent data storage with SQLite  
✅ **Error Handling**: Graceful failure management  
✅ **Best Practices**: Security, performance, and maintainability  

### 🚀 Next Steps

To continue your backend development journey:

1. **Practice Building APIs**: Create a personal project (todo app, blog, inventory system)
2. **Learn Advanced Database**: Explore PostgreSQL, database migrations, ORM libraries (SQLAlchemy)
3. **Authentication & Authorization**: JWT tokens, user sessions, permissions
4. **Testing**: Unit tests, integration tests, API testing
5. **Deployment**: Docker, cloud platforms (AWS, Azure, Google Cloud)
6. **Advanced Topics**: Microservices, caching (Redis), message queues

### 📚 Recommended Resources

- **Flask Documentation**: https://flask.palletsprojects.com/
- **FastAPI Documentation**: https://fastapi.tiangolo.com/
- **SQLAlchemy ORM**: https://www.sqlalchemy.org/
- **Pydantic for Data Validation**: https://pydantic-docs.helpmanual.io/
- **Python Testing with pytest**: https://docs.pytest.org/

### 💡 Project Ideas to Practice

1. **Personal Task Manager API**: CRUD operations for tasks
2. **Simple Blog Backend**: Posts, comments, user management
3. **Inventory System**: Products, categories, stock management
4. **Weather Data API**: Collect and serve weather information
5. **URL Shortener**: Like bit.ly but simpler

**Remember**: The best way to learn backend development is by building real projects. Start small, and gradually add more features!

---

*Happy coding! 🐍✨*